In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.cluster import KMeans
import random
from sklearn.preprocessing import normalize,StandardScaler
from sklearn.decomposition import PCA
from operator import mul

In [2]:
data =pd.read_csv("/Users/steafen/Desktop/data2.csv")
data.drop(['Wenshi','color'],axis=1,inplace=True)

In [3]:
data_K=data[data['Type']==0] # 筛选高钾类型
data_K0=data_K[data_K['Fenghua']==0] # 筛选出高钾未分化
data_K1=data_K[data_K['Fenghua']==1] # 筛选出高钾已分化
data_Pb=data[data['Type']==1]
data_Pb0=data_Pb[data_Pb['Fenghua']==0]
data_Pb1=data_Pb[data_Pb['Fenghua']==1]

In [13]:
# 数据归一化
trainK1_scaled=normalize(data_K1)
trainK1_scaled=pd.DataFrame(trainK1_scaled,columns=data_K1.columns)
traindata_K1=trainK1_scaled.drop(['Type','Fenghua'],axis=1)

trainK0_scaled=normalize(data_K0)
trainK0_scaled=pd.DataFrame(trainK0_scaled,columns=data_K0.columns)
traindata_K0=trainK0_scaled.drop(['Type','Fenghua'],axis=1)

trainPb1_scaled=normalize(data_Pb1)
trainPb1_scaled=pd.DataFrame(trainPb1_scaled,columns=data_Pb1.columns)
traindata_Pb1=trainPb1_scaled.drop(['Type','Fenghua'],axis=1)

trainPb0_scaled=normalize(data_Pb0)
trainPb0_scaled=pd.DataFrame(trainPb0_scaled,columns=data_Pb0.columns)
traindata_Pb0=trainPb0_scaled.drop(['Type','Fenghua'],axis=1)


In [ ]:
# PCA 高钾未风化

In [35]:
pca_K0=PCA(n_components=3)
pca_K0.fit(traindata_K0)
print(pca_K0.explained_variance_ratio_)
print(pca_K0.explained_variance_)
print(pca_K0.n_components)
print(pca_K0.components_)  # 因子载荷矩阵/ 成分矩阵

[0.54210131 0.21925011 0.1078552 ]
[0.00593629 0.0024009  0.00118107]
3
[[ 1.58510792e-01 -9.94606053e-02 -7.72462318e-01 -5.48785938e-01
   1.36729563e-02 -2.01310817e-01 -7.28240995e-02 -1.25533259e-01
  -1.74626292e-02  2.86316404e-02  5.24586029e-02 -4.31364034e-04
   3.76051252e-02 -7.21001491e-03]
 [-7.94843630e-02 -6.25540212e-02 -2.13703578e-01 -2.58496485e-02
   1.28876379e-01  7.25443170e-01  4.55947704e-01  1.32993266e-01
   1.84138885e-02  1.17620062e-01  3.95224506e-01  9.03977334e-03
  -6.70528956e-02 -5.06032639e-03]
 [ 2.83830573e-02 -1.85443752e-01 -4.33988293e-01  6.05942461e-01
  -6.25938049e-02 -3.01076758e-01  2.48320704e-01  4.65652327e-01
   6.47312385e-03  8.20700699e-02 -1.31302782e-01 -8.42390106e-03
  -1.07697603e-01  2.33430350e-02]]


In [83]:
K0_spss=pca_K0.components_/np.sqrt(pca_K0.explained_variance_.reshape(pca_K0.n_components_,1))
K0_spss  # 成分得分系数矩阵---各自因子载荷向量除以各自因子特征值的算数平方根  --代表系数

array([[ 2.05731722e+00, -1.29090274e+00, -1.00258160e+01,
        -7.12271226e+00,  1.77461788e-01, -2.61282028e+00,
        -9.45186584e-01, -1.62930064e+00, -2.26648087e-01,
         3.71611081e-01,  6.80862078e-01, -5.59868918e-03,
         4.88078261e-01, -9.35790406e-02],
       [-1.62216281e+00, -1.27663862e+00, -4.36138613e+00,
        -5.27554565e-01,  2.63018363e+00,  1.48052634e+01,
         9.30524425e+00,  2.71420343e+00,  3.75801278e-01,
         2.40045820e+00,  8.06597013e+00,  1.84488918e-01,
        -1.36845424e+00, -1.03274065e-01],
       [ 8.25888231e-01, -5.39602943e+00, -1.26281612e+01,
         1.76316717e+01, -1.82135019e+00, -8.76071063e+00,
         7.22561861e+00,  1.35495191e+01,  1.88354509e-01,
         2.38806920e+00, -3.82063923e+00, -2.45118089e-01,
        -3.13377737e+00,  6.79234014e-01]])

In [93]:
for i in range(3):
    print(K0_spss[i][[1,2,3,5,6,7]])


[ -1.29090274 -10.02581597  -7.12271226  -2.61282028  -0.94518658
  -1.62930064]
[-1.27663862 -4.36138613 -0.52755457 14.80526345  9.30524425  2.71420343]
[ -5.39602943 -12.62816123  17.6316717   -8.76071063   7.22561861
  13.54951911]


In [104]:
# 最后主成分得分=标准化后的因子得分系数×解释方差比例
K0_score=pca_K0.transform(traindata_K0)    
print('因子得分：',K0_score)
# 因子得分归一化，使其满足正态分布
K0_scaler2=StandardScaler().fit(K0_score)
K0_scaler2=pd.DataFrame(K0_scaler2.transform(K0_score),columns=['FAC1','FAC2','FAC3'])
# 正负号转换
K0_sign=np.sign(K0_spss.sum(axis=1))
# 取正负号
K0_scaler2_sign=K0_scaler2*K0_sign
# 综合得分
K0_rate=pca_K0.explained_variance_ratio_
K0_scaler2_sign['FAC_score']=np.sum(K0_scaler2_sign*K0_rate,axis=1)
print('主成分得分矩阵K0')
print(K0_scaler2_sign) # 主成分得分矩阵

因子得分： [[ 0.00075312 -0.03460555  0.02620342]
 [ 0.11174583 -0.04557964 -0.00094179]
 [-0.05267462 -0.01084866  0.01336897]
 [-0.01609777 -0.00695801  0.01394254]
 [-0.05262557  0.01281977  0.01136228]
 [ 0.05967188  0.07594923 -0.05616618]
 [-0.00789606  0.10466711  0.00171979]
 [-0.09261015 -0.00206211  0.03251467]
 [-0.07610344 -0.00408098 -0.03249501]
 [-0.08163904 -0.04635661 -0.02314202]
 [ 0.08338112 -0.06013088 -0.04751026]
 [ 0.1240947   0.01718634  0.06114359]]
主成分得分矩阵K0
        FAC1      FAC2      FAC3  FAC_score
0  -0.010209 -0.737654  0.796369  -0.081373
1  -1.514844 -0.971578 -0.028623  -1.037305
2   0.714066 -0.231251  0.406307   0.380216
3   0.218224 -0.148317  0.423739   0.131483
4   0.713401  0.273267  0.345320   0.483894
5  -0.808922  1.618939 -1.706991  -0.267673
6   0.107040  2.231091  0.052267   0.552831
7   1.255438 -0.043956  0.988179   0.777517
8   1.031670 -0.086990 -0.987582   0.433681
9   1.106712 -0.988141 -0.703328   0.307443
10 -1.130328 -1.281754 -1.44392

In [82]:
print(traindata_K0)

        SiO2      Na2O       K2O       CaO       MgO     Al2O3     Fe2O3  \
0   0.982232  0.000000  0.141533  0.089539  0.012326  0.055678  0.024651   
1   0.996811  0.000000  0.059431  0.023017  0.000000  0.046491  0.000000   
2   0.967493  0.000000  0.193938  0.092030  0.017403  0.086229  0.033865   
3   0.977957  0.000000  0.143546  0.105693  0.023157  0.095599  0.030580   
4   0.968346  0.000000  0.172189  0.115579  0.027833  0.117938  0.041200   
5   0.977409  0.000000  0.106482  0.000000  0.028607  0.161095  0.034531   
6   0.966181  0.000000  0.124064  0.087394  0.027947  0.162349  0.097571   
7   0.959151  0.000000  0.203663  0.141410  0.000000  0.100125  0.046812   
8   0.961509  0.052023  0.189008  0.126672  0.010158  0.142064  0.007696   
9   0.963979  0.031058  0.214743  0.122309  0.007691  0.091399  0.006212   
10  0.991571  0.000000  0.117551  0.000000  0.019093  0.038061  0.000000   
11  0.992895  0.000000  0.000000  0.060988  0.015797  0.080152  0.030688   

         Cu

In [ ]:
# PCA 高钾已风化

In [97]:
pca_K1=PCA(n_components=2)
pca_K1.fit(traindata_K1)
print(pca_K1.components_)  
print(pca_K1.explained_variance_ratio_)
K1_spss=pca_K1.components_/np.sqrt(pca_K1.explained_variance_.reshape(pca_K1.n_components_,1))
K1_spss # 成分得分矩阵---各自因子载荷向量除以各自因子特征值的算数平方根  --判断主成分代表的变量

[[-2.08788790e-02 -9.08272414e-17 -6.22577129e-02  3.92401033e-01
   2.50068697e-01  8.38734224e-01  2.10399549e-02 -2.73518746e-01
   0.00000000e+00  0.00000000e+00  2.10913050e-02  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-1.94467110e-02 -4.86068669e-17 -3.48321596e-01  1.59573306e-01
  -3.58999205e-02  1.92628533e-01 -4.63868895e-02  8.79260536e-01
   0.00000000e+00  0.00000000e+00  1.97951663e-01  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]
[0.54331407 0.41503859]


array([[-1.68662713e+00, -7.33716065e-15, -5.02927133e+00,
         3.16987433e+01,  2.02009241e+01,  6.77542073e+01,
         1.69963908e+00, -2.20952541e+01,  0.00000000e+00,
         0.00000000e+00,  1.70378722e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [-1.79737751e+00, -4.49252777e-15, -3.21938965e+01,
         1.47486879e+01, -3.31807829e+00,  1.78038431e+01,
        -4.28734462e+00,  8.12663442e+01,  0.00000000e+00,
         0.00000000e+00,  1.82958376e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00]])

In [99]:
for i in range(2):
    print(K1_spss[i][[2,3,4,5,7]])

[ -5.02927133  31.69874335  20.20092406  67.75420731 -22.09525411]
[-32.19389655  14.74868786  -3.31807829  17.80384314  81.26634419]


In [105]:
# 最后主成分得分=标准化后的因子得分系数×解释方差比例
K1_score=pca_K1.transform(traindata_K1)    
print('因子得分：',K1_score)
# 因子得分归一化，使其满足正态分布
K1_scaler2=StandardScaler().fit(K1_score)
K1_scaler2=pd.DataFrame(K1_scaler2.transform(K1_score),columns=['FAC1','FAC2'])
# 正负号转换
K1_sign=np.sign(K1_spss.sum(axis=1))
# 取正负号
K1_scaler2_sign=K1_scaler2*K1_sign
# 综合得分
K1_rate=pca_K1.explained_variance_ratio_
K1_scaler2_sign['FAC_score']=np.sum(K1_scaler2_sign*K1_rate,axis=1)
print('主成分得分矩阵K1')
print(K1_scaler2_sign) # 主成分得分矩阵当做系数

因子得分： [[-0.00366608  0.01939651]
 [-0.00722919 -0.00204803]
 [-0.01177993 -0.01236196]
 [-0.00611239 -0.00248899]
 [ 0.0218105  -0.00587239]
 [ 0.00697709  0.00337487]]
主成分得分矩阵K1
       FAC1      FAC2  FAC_score
0 -0.324418  1.963846   0.638811
1 -0.639723 -0.207358  -0.433632
2 -1.042426 -1.251616  -1.085833
3 -0.540896 -0.252004  -0.398468
4  1.930049 -0.594564   0.801855
5  0.617414  0.341696   0.477267


In [81]:
print(K1_scaler2)
print(traindata_K1)

       FAC1      FAC2
0 -0.324418  1.963846
1 -0.639723 -0.207358
2 -1.042426 -1.251616
3 -0.540896 -0.252004
4  1.930049 -0.594564
5  0.617414  0.341696
       SiO2  Na2O       K2O       CaO       MgO     Al2O3     Fe2O3       CuO  \
0  0.999013   0.0  0.000000  0.011540  0.000000  0.021354  0.001833  0.034943   
1  0.999662   0.0  0.006207  0.006523  0.000000  0.013887  0.003367  0.016307   
2  0.999823   0.0  0.009505  0.002170  0.000000  0.008369  0.002686  0.008679   
3  0.999579   0.0  0.010707  0.007633  0.000000  0.015478  0.003074  0.017492   
4  0.998980   0.0  0.008005  0.017957  0.006923  0.037861  0.003786  0.005950   
5  0.999360   0.0  0.000000  0.010132  0.005820  0.027053  0.002156  0.016599   

   PbO  BaO      P2O5  SrO  SnO2  SO2  
0  0.0  0.0  0.006579  0.0   0.0  0.0  
1  0.0  0.0  0.003682  0.0   0.0  0.0  
2  0.0  0.0  0.000000  0.0   0.0  0.0  
3  0.0  0.0  0.001590  0.0   0.0  0.0  
4  0.0  0.0  0.002272  0.0   0.0  0.0  
5  0.0  0.0  0.003880  0.0   0.0  0.0 

In [58]:
# 铅钡未风化

In [85]:
pca_Pb0=PCA(n_components=2)
pca_Pb0.fit(traindata_Pb0)
print(pca_Pb0.components_)
print(pca_Pb0.explained_variance_ratio_)
Pb0_spss=pca_Pb0.components_/np.sqrt(pca_Pb0.explained_variance_.reshape(pca_Pb0.n_components_,1))
Pb0_spss # 成分得分矩阵---各自因子载荷向量除以各自因子特征值的算数平方根  --判断主成分代表的变量

[[-0.55133646 -0.00350205  0.01233836  0.07465498  0.00116028  0.02964334
   0.0199475   0.08449069  0.73262649  0.38032409  0.0009367   0.01500088
   0.00840304 -0.01375087]
 [-0.07520855 -0.0275685   0.00534957 -0.09155801 -0.02155311  0.07501542
  -0.01566936  0.26535167 -0.49554627  0.79917586  0.15502499  0.00285657
  -0.0080996   0.00828296]]
[0.65283332 0.29999426]


array([[-2.39543446e+00, -1.52156411e-02,  5.36074351e-02,
         3.24359317e-01,  5.04114191e-03,  1.28793718e-01,
         8.66674393e-02,  3.67093269e-01,  3.18309938e+00,
         1.65242371e+00,  4.06976909e-03,  6.51755024e-02,
         3.65093432e-02, -5.97444926e-02],
       [-4.82035984e-01, -1.76695471e-01,  3.42871567e-02,
        -5.86824993e-01, -1.38140873e-01,  4.80798162e-01,
        -1.00429991e-01,  1.70072498e+00, -3.17611686e+00,
         5.12217749e+00,  9.93605492e-01,  1.83087122e-02,
        -5.19129963e-02,  5.30881840e-02]])

In [106]:
# 最后主成分得分=标准化后的因子得分系数×解释方差比例
Pb0_score=pca_Pb0.transform(traindata_Pb0)    
print('因子得分：',Pb0_score)
# 因子得分归一化，使其满足正态分布
Pb0_scaler2=StandardScaler().fit(Pb0_score)
Pb0_scaler2=pd.DataFrame(Pb0_scaler2.transform(Pb0_score),columns=['FAC1','FAC2'])
# 正负号转换
Pb0_sign=np.sign(Pb0_spss.sum(axis=1))
# 取正负号
Pb0_scaler2_sign=Pb0_scaler2*Pb0_sign
# 综合得分
Pb0_rate=pca_Pb0.explained_variance_ratio_
Pb0_scaler2_sign['FAC_score']=np.sum(Pb0_scaler2_sign*Pb0_rate,axis=1)
print('主成分得分矩阵Pb0')
print(Pb0_scaler2_sign) # 主成分得分矩阵当做系数

因子得分： [[ 0.01133101  0.40151871]
 [ 0.38586555  0.22495814]
 [ 0.36871137 -0.16096913]
 [ 0.31904372 -0.14347224]
 [-0.22747822 -0.0482867 ]
 [-0.19942357 -0.05182413]
 [-0.26280347 -0.03816752]
 [-0.15430582 -0.06110534]
 [-0.15203915  0.04208442]
 [-0.17066699  0.04593714]
 [-0.02503193 -0.03159465]
 [-0.001231   -0.0529604 ]
 [ 0.1080285  -0.1261183 ]]
主成分得分矩阵Pb0
        FAC1      FAC2  FAC_score
0   0.051241  2.678546   0.837000
1   1.744957  1.500704   1.589368
2   1.667382 -1.073831   0.766380
3   1.442776 -0.957109   0.654765
4  -1.028699 -0.322122  -0.768204
5  -0.901831 -0.345721  -0.692459
6  -1.188447 -0.254617  -0.852241
7  -0.697800 -0.407636  -0.577836
8  -0.687550  0.280747  -0.364633
9  -0.771788  0.306448  -0.411916
10 -0.113199 -0.210769  -0.137130
11 -0.005567 -0.353301  -0.109622
12  0.488525 -0.841340   0.066528


In [101]:
for i in range(2):
    print(Pb0_spss[i][[0,7,8,9]])

[-2.39543446  0.36709327  3.18309938  1.65242371]
[-0.48203598  1.70072498 -3.17611686  5.12217749]


In [61]:
# 铅钡已风化

In [86]:
pca_Pb1=PCA(n_components=2)
pca_Pb1.fit(traindata_Pb1)
print(pca_Pb1.components_)
print(pca_Pb1.explained_variance_ratio_)
Pb1_spss=pca_Pb1.components_/np.sqrt(pca_Pb1.explained_variance_.reshape(pca_Pb1.n_components_,1))
Pb1_spss # 成分得分矩阵---各自因子载荷向量除以各自因子特征值的算数平方根  --判断主成分代表的变量

[[ 7.29448527e-01  5.22165954e-02  1.88135331e-03 -3.36036443e-02
   4.86015021e-03  9.59675332e-02  8.67843905e-04 -2.97006508e-02
  -6.40132113e-01 -1.69373180e-01 -1.02358564e-01 -6.75838997e-03
   2.77984600e-03 -7.03006645e-02]
 [-1.75636712e-01  1.56890781e-02 -3.84895932e-04 -1.33028992e-02
  -2.08501695e-02 -3.65236922e-02 -2.38826595e-02  1.59646474e-01
  -4.51486608e-01  8.17347099e-01 -2.94366686e-02  2.40373107e-03
   3.33783851e-05  2.61024254e-01]]
[0.67621936 0.23546893]


array([[ 2.05311310e+00,  1.46969350e-01,  5.29527577e-03,
        -9.45811523e-02,  1.36794272e-02,  2.70111176e-01,
         2.44264211e-03, -8.35957480e-02, -1.80172223e+00,
        -4.76719441e-01, -2.88099435e-01, -1.90222318e-02,
         7.82418226e-03, -1.97868951e-01],
       [-8.37742426e-01,  7.48329108e-02, -1.83585566e-03,
        -6.34514443e-02, -9.94500035e-02, -1.74208719e-01,
        -1.13914209e-01,  7.61473063e-01, -2.15347624e+00,
         3.89853770e+00, -1.40405420e-01,  1.14651856e-02,
         1.59206404e-04,  1.24501928e+00]])

In [103]:
for i in range(2):
    print(Pb1_spss[i][[0,8,9,13]])

[ 2.0531131  -1.80172223 -0.47671944 -0.19786895]
[-0.83774243 -2.15347624  3.8985377   1.24501928]


In [107]:
# 最后主成分得分=标准化后的因子得分系数×解释方差比例
Pb1_score=pca_Pb1.transform(traindata_Pb1)    
print('因子得分：',Pb1_score)
# 因子得分归一化，使其满足正态分布
Pb1_scaler2=StandardScaler().fit(Pb1_score)
Pb1_scaler2=pd.DataFrame(Pb1_scaler2.transform(Pb1_score),columns=['FAC1','FAC2'])
# 正负号转换
Pb1_sign=np.sign(Pb1_spss.sum(axis=1))
# 取正负号
Pb1_scaler2_sign=Pb1_scaler2*Pb1_sign
# 综合得分
Pb1_rate=pca_Pb1.explained_variance_ratio_
Pb1_scaler2_sign['FAC_score']=np.sum(Pb1_scaler2_sign*Pb1_rate,axis=1)
print('主成分得分矩阵Pb1')
print(Pb1_scaler2_sign) # 主成分得分矩阵当做系数

因子得分： [[-3.02274577e-02 -2.37224273e-01]
 [-1.68701390e-01  4.62166022e-01]
 [-4.76580035e-01  5.14731324e-01]
 [ 1.37173453e-01  1.20398198e-01]
 [-1.06786291e-01 -1.45198374e-01]
 [ 4.92501082e-01  1.12277607e-01]
 [ 2.91363715e-01 -6.39782130e-02]
 [-1.83867201e-01  4.65900328e-01]
 [-4.54589216e-01  6.17657431e-01]
 [ 5.75829051e-01 -4.79285340e-03]
 [ 6.25059196e-01 -3.77020680e-03]
 [-5.68779628e-02 -8.77967719e-02]
 [ 4.33788473e-02 -5.68331042e-02]
 [-1.16437111e-01 -1.02256245e-01]
 [-2.85491015e-01 -1.61150078e-01]
 [-4.37220368e-01 -1.72889314e-01]
 [-3.08828158e-01 -8.35411114e-02]
 [ 4.19759793e-01  5.61651428e-02]
 [ 4.45775703e-01  7.37006127e-02]
 [-4.60367476e-01 -1.33170120e-01]
 [-2.48030019e-01 -1.87417500e-01]
 [ 5.96400256e-01  2.78921142e-02]
 [ 5.29837423e-01  3.86874059e-02]
 [-2.45744750e-02 -1.10602631e-01]
 [ 4.40938409e-01 -5.17622256e-02]
 [-3.27261488e-01  3.38742616e-04]
 [ 2.45932450e-01 -7.48136351e-02]
 [-1.66607043e-01 -8.46322407e-02]
 [-2.87524636e

In [64]:
# 因子得分公式

In [66]:
Pb0_scaler2_sign

,FAC1,FAC2,FAC_score
0,0.051241,2.678546,0.837000
1,1.744957,1.500704,1.589368
2,1.667382,-1.073831,0.766380
3,1.442776,-0.957109,0.654765
4,-1.028699,-0.322122,-0.768204
5,-0.901831,-0.345721,-0.692459
6,-1.188447,-0.254617,-0.852241
7,-0.697800,-0.407636,-0.577836
8,-0.687550,0.280747,-0.364633
9,-0.771788,0.306448,-0.411916


In [69]:
# 高钾未风化
Pb0_w=Pb0_scaler2_sign.loc[:,'FAC1'].tolist()

[0.05124094661896188,
 1.7449566584380054,
 1.667382230079898,
 1.4427757309885698,
 -1.02869932508745,
 -0.9018309156872227,
 -1.1884467514299712,
 -0.6977999657890344,
 -0.6875496575517943,
 -0.7717882550881139,
 -0.11319907938039979,
 -0.005566821112149563,
 0.48852520500070123]

In [72]:
def yinzi_Pb0(x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12):
    Pb0_x=[0,x2,x3,x4,0,x6,x7,x8,0,0,0,0]
    return mul(Pb0_w,Pb0_x)